In [1]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [2]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(file_path="raw_data/new_format_test.csv", encoding='latin-1')
docs = loader.load()

In [11]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

In [12]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

embeddings = OpenAIEmbeddings(openai_api_key=api_key)
retriever = FAISS.from_documents(docs, embeddings).as_retriever()


In [ ]:
# v2

In [24]:
from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [27]:
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, chain_type_kwargs=chain_type_kwargs)

In [31]:
query = "How active is the tour?"
result = qa.run(query)
print(result.strip())

The tour is light and suitable for most fitness levels. There is some light walking and hiking, but nothing too challenging.


In [23]:
from langchain.prompts import PromptTemplate

prompt_template = """
Question: {question}
Context: 
{context}
Answer:
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["question", "context"])

qa_chain = load_qa_chain(llm=OpenAI(temperature=0), chain_type="stuff", prompt=prompt)

qa = RetrievalQA(
    combine_documents_chain=qa_chain, 
    retriever=retriever,
    return_source_documents=True
)


In [ ]:
answer = qa.get_answer(question)

In [ ]:
print(result["answer"]) 
print(result["docs"])